In [1]:
!nvidia-smi

Wed Jun 21 16:13:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.156.00   Driver Version: 450.156.00   CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      On   | 00000000:07:00.0 Off |                    0 |
| N/A   30C    P0    59W / 400W |  20957MiB / 40537MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  A100-SXM4-40GB      On   | 00000000:0F:00.0 Off |                    0 |
| N/A   

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [3]:
!pip install -qq transformers
!pip install -qq scikit-multilearn

In [4]:
import random
import numpy as np
import nlpaug.augmenter.word as naw
import torch
import torch.nn.functional as F
import torch.nn as nn

from transformers import AutoTokenizer, AutoModel, get_constant_schedule_with_warmup, BertForPreTraining
from transformers import TrainingArguments, Trainer, AutoConfig
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [5]:
import pandas as pd
import re
import numpy as np
import warnings
import string
warnings.filterwarnings("ignore", 'This pattern has match groups')
import matplotlib.pyplot as plt
import matplotlib

In [6]:
seed_val = 1906350912
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)

In [7]:
if torch.cuda.is_available():
  torch.cuda.manual_seed_all(seed_val)

In [8]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('Tersedia sejumlah %d GPU(s).' % torch.cuda.device_count())
    print('GPU yang akan digunakan:', torch.cuda.get_device_name(0))
else:
    print('Tidak mendukung GPU; hanya CPU')
    device = torch.device("cpu")

Tersedia sejumlah 1 GPU(s).
GPU yang akan digunakan: A100-SXM4-40GB


In [9]:
df_train = pd.read_csv(f"Dataset/Human_Annotated.csv", index_col="ID")
df_train.fillna("", inplace = True)
df_test = pd.read_csv(f"Dataset/Gold_Standard.csv", index_col="ID")
df_test.fillna("", inplace = True)
categories = ['Anak', 'Bedah', 'Gigi', 'Gizi', 'Jantung', 'Jiwa',
       'Kandungan', 'Kulit dan Kelamin', 'Mata', 'Paru', 'Penyakit Dalam',
       'Saraf', 'THT', 'Tulang', 'Urologi']

In [10]:
df_train_umumnt = df_train.drop(columns=["Umum"])
df_train_umumnt["COUNT CLASS"] = df_train_umumnt.drop(columns=["JUDUL", "ISI"]).sum(axis=1)
df_train = df_train_umumnt[df_train_umumnt["COUNT CLASS"] > 0].drop(columns=["COUNT CLASS"])

In [11]:
df_test['Count'] = df_test.drop(columns=['JUDUL', 'ISI']).values.sum(axis=1)
df_test = df_test[df_test['Count'] <= 3].drop(columns=['Count'])

In [12]:
df_test_umumnt = df_test.drop(columns=["Umum"])
df_test_umumnt["COUNT CLASS"] = df_test_umumnt.drop(columns=["JUDUL", "ISI"]).sum(axis=1)
df_test = df_test_umumnt[df_test_umumnt["COUNT CLASS"] > 0].drop(columns=["COUNT CLASS"])

In [13]:
code = ['DS', 'AD', 'DH', 'TD', 'KD']
for i in code:
  df_train[i] = df_train.index.map(lambda x: 1 if x[:2] == i else 0)

X = df_train[['JUDUL', 'ISI']].values
y = df_train.drop(columns=['JUDUL', 'ISI']).values

from skmultilearn.model_selection import iterative_train_test_split
X_train, y_train, X_val, y_val = iterative_train_test_split(X, y,test_size = 0.2)
y_train = y_train[:,:-5]
y_val = y_val[:,:-5]

In [14]:
df_test = df_test[["JUDUL", "ISI"] + categories]
X_test = df_test[['JUDUL', 'ISI']].values
y_test = df_test.drop(columns=['JUDUL', 'ISI']).values

In [15]:
#--------------------------------
#  Some Parameters
#--------------------------------
MAX_SEQ_LENGTH = 256
BATCH_SIZE = 8
WARMUP_PROPORTION = 0.1
EPOCH = 100
REGULARIZATION = 0.02
MODEL_EPOCH = 7
MODEL_NAME = f"Hendrico/dapt-indobert-large-{MODEL_EPOCH}"
LEARNING_RATE = 1e-5
PATH = f"Model/dapt-indobert-large-{MODEL_EPOCH}-finetuned.pt"
BASE = "indobenchmark/indobert-large-p1"

In [17]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [18]:
categories = ['Anak', 'Bedah', 'Gigi', 'Gizi', 'Jantung', 'Jiwa',
       'Kandungan', 'Kulit dan Kelamin', 'Mata', 'Paru', 'Penyakit Dalam',
       'Saraf', 'THT', 'Tulang', 'Urologi']
label2idx = {k: v for v, k in enumerate(categories)}
idx2label = {v: k for v, k in enumerate(categories)}

In [19]:
def create_data_loader(x, y):
  input_ids = []
  input_att_mask = []
  input_label = []
  for instance, label in zip(x, y):
    encoded_sent = tokenizer.encode(instance, \
                                  add_special_tokens = True, \
                                  max_length = MAX_SEQ_LENGTH, \
                                  padding = "max_length", \
                                  truncation = True)
    input_ids.append(encoded_sent)
    input_att_mask.append([int(token_id > 0) for token_id in encoded_sent])
    input_label.append(label)
  
  # ubah ke Torch's Tensor
  input_ids = torch.tensor(input_ids)
  input_att_mask = torch.tensor(input_att_mask)
  input_label = torch.tensor(input_label)

  # gunakan Torch's Dataset
  dataset = TensorDataset(input_ids, input_att_mask, input_label)

  # mengembalikan sebuah abstraksi DataLoader dari Torch
  return DataLoader(dataset,
#                     sampler = SequentialSampler(dataset), 
                    sampler = RandomSampler(dataset), 
                    batch_size = BATCH_SIZE)

In [20]:
train_instances = create_data_loader(X_train, y_train)
valid_instances = create_data_loader(X_val, y_val)
test_instances = create_data_loader(X_test, y_test)
dataset = {"Train" : train_instances, "Val" : valid_instances, "Test" : test_instances}

/tmp/ipykernel_209735/1514995109.py:18: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:207.)
  input_label = torch.tensor(input_label)


In [21]:
# Classifier adalah layer on top of BERT yang digunakan untuk klasifikasi
# inputnya adalah vektor CLS dari BERT
class Classifier(nn.Module):
    def __init__(self, input_size = 512, output_size = 3, dropout_rate = 0.1):
        super(Classifier, self).__init__()
        layers = []
        layers.append(nn.Linear(input_size, output_size))
        layers.append(nn.LeakyReLU(0.2, inplace = True))
        layers.append(nn.Dropout(dropout_rate))
        layers.append(nn.Sigmoid())
        self.layers = nn.Sequential(*layers)

    def forward(self, input):
        return self.layers(input)


In [22]:
transformer = None
classifier = None
def create_model(transformer, classifier):
  if classifier != None:
    del transformer, classifier
    torch.cuda.empty_cache()
  config = AutoConfig.from_pretrained(MODEL_NAME)
  transformer = AutoModel.from_pretrained(MODEL_NAME)
  classifier = Classifier(input_size = config.hidden_size, output_size = len(categories))
  if torch.cuda.is_available():    
    classifier.cuda()
    transformer.cuda()
  return transformer, classifier
transformer, classifier = create_model(transformer, classifier)

Some weights of the model checkpoint at Hendrico/cpt-indobert-large-8 were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [23]:
def get_param(model, cls):
    model_vars = [p for p in model.parameters()]
    cls_vars = [p for p in cls.parameters()]
    return model_vars + cls_vars

In [25]:
# training
from tqdm import tqdm
from time import sleep
import copy
from sklearn.metrics import classification_report, accuracy_score, recall_score, precision_score

def train_model(model, cls, optimizer, scheduler, num_epochs=25, last_epoch=0, last_acc=0):

  best_acc = last_acc
  max_epoch = last_epoch + num_epochs
  loss = torch.tensor([0])
  for epoch_i in range(last_epoch, max_epoch):
    print(f"Epoch {epoch_i + 1} / {max_epoch}")
    pred_labels = []
    true_labels = []
    
    if np.isnan(loss.item()):
        break
        
    for phase in ["Train", "Val"]:
      instance = dataset[phase]
      if phase == "Train":
        model.train()
        cls.train()
      else:
        model.eval()
        cls.eval()
      # Reset the total loss for this epoch.
      loss_track = 0
      
      # Untuk setiap batch di training data
      pbar = tqdm(enumerate(instance), desc=phase, total=len(instance))
      for step, batch in pbar:

        batch_input_ids = batch[0].to(device)
        batch_input_att_mask = batch[1].to(device)
        batch_label = batch[2].to(device)

        if phase == "Train":
            # Encode real data in the Transformer
            bert_outputs = transformer(batch_input_ids, attention_mask = batch_input_att_mask)
            cls_hidden_states = bert_outputs.pooler_output
            probs = classifier(cls_hidden_states)

            # Categorical Cross-Entropy
            loss = -torch.mean(torch.sum(batch_label * torch.log(probs) + (1 - batch_label) * torch.log(1 - probs), dim = 1))
            
            loss.to(device)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_track += loss.item()
            scheduler.step()
            

        elif phase == "Val":
            with torch.no_grad(): 
                bert_outputs = transformer(batch_input_ids, attention_mask = batch_input_att_mask)
                cls_hidden_states = bert_outputs.pooler_output
                probs = classifier(cls_hidden_states)

                # Categorical Cross-Entropy
                loss = -torch.mean(torch.sum(batch_label * torch.log(probs) + (1 - batch_label) * torch.log(1 - probs), dim = 1))

                # akumulasi loss di validation set
                loss_track += loss.item()
            pred_l = []
            for prob in probs:
                pred = [1 if x >= 0.5 else 0 for x in prob]
                if (np.sum(pred) == 0):
                    _, idx = torch.max(prob, dim = 0)
                    pred[idx] = 1
                pred_l.append(pred)
            pred_labels += pred_l
            true_l = batch_label
            true_labels += true_l.detach().cpu()

        # pbar.set_description(f"Average loss : {(loss_track / len(instance)):.2f}")
        pbar.set_postfix(loss = str(f'{(loss_track / len(instance)):.2f}'))
        sleep(0.1)

  # accuracy
    pred_labels = np.array(pred_labels)
    true_labels = torch.stack(true_labels).numpy()
    val_accuracy = accuracy_score(true_labels, pred_labels)
    val_precision = precision_score(true_labels, pred_labels, average='micro')
    val_recall = recall_score(true_labels, pred_labels, average='micro')
    print(f"Accuracy: {val_accuracy}, Precision (micro): {val_precision}, Recall (micro): {val_recall}")

    if (val_accuracy > best_acc):
      
      torch.save({
                  'best_model_sd' : model.state_dict(),
                  'best_cls_sd' : cls.state_dict(),
                  'best_optimizer_sd' : optimizer.state_dict(),
                  'best_scheduler_sd' : scheduler.state_dict(),
                  'best_epoch' : epoch_i,
                  'best_acc' : val_accuracy
                  }, PATH)
      best_acc = val_accuracy

  checkpoint = torch.load(PATH)
  model.load_state_dict(checkpoint['best_model_sd'])
  cls.load_state_dict(checkpoint['best_cls_sd'])
  optimizer.load_state_dict(checkpoint['best_optimizer_sd'])
  scheduler.load_state_dict(checkpoint['best_scheduler_sd'])
  best_epoch = checkpoint['best_epoch']
  best_acc = checkpoint['best_acc']
  return {'model' : model, 'cls' : cls, 'optim' : optimizer, 
          'scheduler' : scheduler, 'best_acc' : best_acc, "best_epoch" : best_epoch}  

In [29]:
%%time
transformer, classifier = create_model(transformer, classifier)
optimizer = torch.optim.AdamW(get_param(transformer, classifier), lr = LEARNING_RATE, weight_decay = REGULARIZATION)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
info = train_model(transformer, classifier, optimizer, scheduler, num_epochs=100, last_epoch=0, last_acc=0)

Some weights of the model checkpoint at Hendrico/cpt-indobert-large-6 were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch 1 / 100


Val: 100%|██████████| 40/40 [00:05<00:00,  7.30it/s, loss=6.88]
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.16293929712460065, Precision (macro): 0.012992545260915868, Recall (macro): 0.06666666666666667
Epoch 2 / 100


Val: 100%|██████████| 40/40 [00:05<00:00,  7.30it/s, loss=5.82]


...

Accuracy: 0.7028753993610224, Precision (macro): 0.6875133232597743, Recall (macro): 0.660364350742065
Epoch 99 / 100


Val: 100%|██████████| 40/40 [00:05<00:00,  7.32it/s, loss=1.87]


Accuracy: 0.7060702875399361, Precision (macro): 0.6895942282469798, Recall (macro): 0.6630076345138277
Epoch 100 / 100


Val: 100%|██████████| 40/40 [00:05<00:00,  7.28it/s, loss=1.88]


Accuracy: 0.7028753993610224, Precision (macro): 0.68582844165686, Recall (macro): 0.6614572469169284
CPU times: user 33min 3s, sys: 7min 50s, total: 40min 54s
Wall time: 1h 7min 43s


In [28]:
checkpoint = torch.load(PATH)
transformer.load_state_dict(checkpoint['best_model_sd'])
classifier.load_state_dict(checkpoint['best_cls_sd'])
# optimizer.load_state_dict(checkpoint['best_optimizer_sd'])
# scheduler.load_state_dict(checkpoint['best_scheduler_sd'])
best_epoch = checkpoint['best_epoch']
best_acc = checkpoint['best_acc']
print(f"Acc : {best_acc}, Epoch : {best_epoch}")

Acc : 0.7795527156549521, Epoch : 30


In [30]:
from sklearn.metrics import classification_report, accuracy_score, jaccard_score, \
hamming_loss, cohen_kappa_score, f1_score, precision_score, recall_score
import math
def eval_all(y_human, y_machine, categories):
  print("================ Classification Report ================")
  print(classification_report(y_human, y_machine, target_names=categories))
  print("================ Multi Label Score ================")
  eval_multi_label(y_human, y_machine, categories)

def eval_multi_label(y_test, prediction, categories):
    # Butuh informasi lebih lengkap? silakan disimak di bawah ini
    print('Accuracy             :', accuracy_score(y_test, prediction))
    print('Precision            :', precision_score(y_test, prediction, average='micro'))
    print('Recall               :', recall_score(y_test, prediction, average='micro'))
    print('F1 Score             :', f1_score(y_test, prediction, average='micro'))
    print('Hamming Loss         :', hamming_loss(y_test, prediction))
    print('Jaccard Score')
    js = jaccard_score(y_test, prediction, average=None)
    for i, j in zip(categories, js):
      print(f"{i:<20} : {j}")
    print('Jaccard Score Macro Average:', jaccard_score(y_test, prediction, average='macro'))
    print('Jaccard Score Micro Average:', jaccard_score(y_test, prediction, average='micro'))

In [31]:
def evaluate_dataset(model, cls, instance):
    cls.eval()
    model.eval()

    loss_track = 0
    pred_labels = []
    true_labels = []

    # Untuk setiap batch di validation data
    pbar = tqdm(enumerate(instance), desc="Evaluate", total=len(instance))
    for step, batch in pbar:

        # Unpack sebuah batch dari DataLoader
        batch_input_ids = batch[0].to(device)
        batch_input_att_mask = batch[1].to(device)
        batch_label = batch[2].to(device)

        # jangan track gradient! ini sedang evaluasi, bukan training
        with torch.no_grad(): 
            bert_outputs = transformer(batch_input_ids, attention_mask = batch_input_att_mask)
            cls_hidden_states = bert_outputs.pooler_output
            probs = classifier(cls_hidden_states)

        # akumulasi prediksi
        pred_l = []
        for prob in probs:
            pred = [1 if x >= 0.5 else 0 for x in prob]
            if (np.sum(pred) == 0):
                _, idx = torch.max(prob, dim = 0)
                pred[idx] = 1
            pred_l.append(pred)
        pred_labels += pred_l
        true_l = batch_label
        true_labels += true_l.detach().cpu()

    pred_labels = np.array(pred_labels)
    true_labels = torch.stack(true_labels).numpy()
    eval_all(true_labels, pred_labels, categories)
    print("")

In [35]:
classifier.eval()
transformer.eval()
judul = ''
isi = 'Saya belakangan ini sakit pilek. Apakah saya kena covid dok?'
encoded_sent = tokenizer.encode([judul, isi], \
                                  add_special_tokens = True, \
                                  max_length = MAX_SEQ_LENGTH, \
                                  padding = "max_length", \
                                  truncation = True)
input_ids = torch.tensor([encoded_sent]).cuda()
input_att_mask = torch.tensor([[int(token_id > 0) for token_id in encoded_sent]]).cuda()

with torch.no_grad(): 
    bert_outputs = transformer(input_ids, attention_mask = input_att_mask)
    cls_hidden_states = bert_outputs.pooler_output
    probs = classifier(cls_hidden_states)
    print(probs)
pred_l = []
for prob in probs:
    pred = [1 if x >= 0.5 else 0 for x in prob]
    if (np.sum(pred) == 0):
        _, idx = torch.max(prob, dim = 0)
        pred[idx] = 1
    pred_l.append(pred)
print(pred_l)
pred_lst = []
for i, j in enumerate(pred_l[0]):
  if j:
    pred_lst.append(idx2label[i])
print(judul)
print(isi)
print(pred_lst)

tensor([[0.0953, 0.0599, 0.0451, 0.0623, 0.0514, 0.0651, 0.0909, 0.0781, 0.0654,
         0.0422, 0.0978, 0.0701, 0.9390, 0.0527, 0.0421]], device='cuda:0')
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]]

Saya belakangan ini sakit pilek. Apakah saya kena covid dok?
['THT']


In [35]:
evaluate_dataset(transformer, classifier, train_instances)

Evaluate: 100%|██████████| 156/156 [00:04<00:00, 31.44it/s]

================ Classification Report ================
                   precision    recall  f1-score   support

             Anak       1.00      0.99      0.99       148
            Bedah       0.99      0.93      0.96       116
             Gigi       1.00      0.93      0.97        30
             Gizi       1.00      0.84      0.91        88
          Jantung       0.44      0.13      0.20        31
             Jiwa       1.00      0.92      0.96        90
        Kandungan       1.00      0.97      0.98       244
Kulit dan Kelamin       1.00      0.94      0.97       172
             Mata       1.00      0.98      0.99        65
             Paru       0.00      0.00      0.00        17
   Penyakit Dalam       0.97      0.91      0.94       197
            Saraf       0.88      0.55      0.67        51
              THT       1.00      0.92      0.96       125
           Tulang       0.97      0.87      0.92        39
          Urologi       0.94      0.55      0.69        31


/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [36]:
evaluate_dataset(transformer, classifier, valid_instances)

Evaluate: 100%|██████████| 40/40 [00:01<00:00, 31.90it/s]

================ Classification Report ================
                   precision    recall  f1-score   support

             Anak       0.92      0.95      0.93        37
            Bedah       0.70      0.90      0.79        29
             Gigi       1.00      0.86      0.92         7
             Gizi       0.64      0.70      0.67        23
          Jantung       0.00      0.00      0.00         7
             Jiwa       0.80      0.73      0.76        22
        Kandungan       0.86      0.80      0.83        61
Kulit dan Kelamin       0.93      0.91      0.92        43
             Mata       0.94      1.00      0.97        15
             Paru       0.00      0.00      0.00         4
   Penyakit Dalam       0.69      0.55      0.61        49
            Saraf       0.67      0.17      0.27        12
              THT       0.96      0.84      0.90        31
           Tulang       0.80      0.89      0.84         9
          Urologi       0.75      0.38      0.50         8

In [37]:
evaluate_dataset(transformer, classifier, test_instances)

Evaluate: 100%|██████████| 340/340 [00:10<00:00, 30.95it/s]


================ Classification Report ================
                   precision    recall  f1-score   support

             Anak       0.80      0.80      0.80       286
            Bedah       0.57      0.46      0.51       237
             Gigi       0.93      0.75      0.83       232
             Gizi       0.76      0.70      0.73       240
          Jantung       0.00      0.00      0.00        72
             Jiwa       0.72      0.51      0.60       144
        Kandungan       0.82      0.83      0.83       393
Kulit dan Kelamin       0.84      0.62      0.71       419
             Mata       0.97      0.85      0.91       215
             Paru       0.00      0.00      0.00        19
   Penyakit Dalam       0.55      0.53      0.54       426
            Saraf       0.30      0.43      0.35        95
              THT       0.75      0.68      0.71       235
           Tulang       0.74      0.47      0.57       132
          Urologi       0.30      0.17      0.22       126

# ASO

In [27]:
def aso_pred(model, cls, instance):
    cls.eval()
    model.eval()

    loss_track = 0
    pred_labels = []
    true_labels = []

    # Untuk setiap batch di validation data
    pbar = tqdm(enumerate(instance), desc="Evaluate", total=len(instance))
    for step, batch in pbar:

        # Unpack sebuah batch dari DataLoader
        batch_input_ids = batch[0].to(device)
        batch_input_att_mask = batch[1].to(device)
        batch_label = batch[2].to(device)

        # jangan track gradient! ini sedang evaluasi, bukan training
        with torch.no_grad(): 
            bert_outputs = transformer(batch_input_ids, attention_mask = batch_input_att_mask)
            cls_hidden_states = bert_outputs.pooler_output
            probs = classifier(cls_hidden_states)

        # akumulasi prediksi
        pred_l = []
        for prob in probs:
            pred = [1 if x >= 0.5 else 0 for x in prob]
            if (np.sum(pred) == 0):
                _, idx = torch.max(prob, dim = 0)
                pred[idx] = 1
            pred_l.append(pred)
        pred_labels += pred_l
        true_l = batch_label
        true_labels += true_l.detach().cpu()

    pred_labels = np.array(pred_labels)
    true_labels = torch.stack(true_labels).numpy()
    return pred_labels, true_labels

def aso_eval(prediction, y_test):
    return accuracy_score(y_test, prediction), f1_score(y_test, prediction, average='micro', zero_division=0), hamming_loss(y_test, prediction)

In [28]:
import csv  
import os

# Specifying path

path = 'Result/deep.csv'

# Checking whether the specified path exists

if not os.path.exists(path):
  print("gada")
  f = open(path, "w+")
  writer = csv.writer(f)
  writer.writerow(['Id', "Learning Method", 'Model', 'Accuracy', 'F1-Score', 'Hamming Loss'])
  f.close()
else:
  print('ada')

def record_result(row):
  f = open(path, "a")
  writer = csv.writer(f)
  writer.writerow(row)
  f.close()

ada


In [ ]:
%%time
transformer, classifier = create_model(transformer, classifier)
optimizer = torch.optim.AdamW(get_param(transformer, classifier), lr = LEARNING_RATE, weight_decay = REGULARIZATION)

num_train_instances = len(train_instances)
num_train_steps = int(num_train_instances / BATCH_SIZE * EPOCH)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)
scheduler = get_constant_schedule_with_warmup(optimizer, num_warmup_steps = num_warmup_steps)

info = train_model(transformer, classifier, optimizer, scheduler, num_epochs=EPOCH, last_epoch=0, last_acc=0)

y_pred, y_test = aso_pred(transformer, classifier, test_instances)
acc, f1, hamm = aso_eval(y_pred, y_test)

record_result([f"daptE6-1", "semi-supervised", "dapt", acc, f1, hamm])

evaluate_dataset(transformer, classifier, test_instances)

Some weights of the model checkpoint at Hendrico/cpt-indobert-large-8 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch 1 / 100


Val: 100%|██████████| 40/40 [00:07<00:00,  5.44it/s, loss=8.11]


Accuracy: 0.1597444089456869, Precision (micro): 0.19169329073482427, Recall (micro): 0.16806722689075632
Epoch 2 / 100


Val: 100%|██████████| 40/40 [00:07<00:00,  5.61it/s, loss=6.28]


...

Val: 100%|██████████| 40/40 [00:11<00:00,  3.58it/s, loss=1.54]


Accuracy: 0.7476038338658147, Precision (micro): 0.8211382113821138, Recall (micro): 0.8487394957983193
Epoch 84 / 100


Val: 100%|██████████| 40/40 [00:10<00:00,  3.88it/s, loss=1.64]


Accuracy: 0.744408945686901, Precision (micro): 0.8162162162162162, Recall (micro): 0.84593837535014
Epoch 85 / 100


Train:   2%|▏         | 3/156 [00:01<00:52,  2.90it/s, loss=0.03]

In [ ]:
%%time
transformer, classifier = create_model(transformer, classifier)
optimizer = torch.optim.AdamW(get_param(transformer, classifier), lr = LEARNING_RATE, weight_decay = REGULARIZATION)

num_train_instances = len(train_instances)
num_train_steps = int(num_train_instances / BATCH_SIZE * EPOCH)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)
scheduler = get_constant_schedule_with_warmup(optimizer, num_warmup_steps = num_warmup_steps)

info = train_model(transformer, classifier, optimizer, scheduler, num_epochs=EPOCH, last_epoch=0, last_acc=0)

y_pred, y_test = aso_pred(transformer, classifier, test_instances)
acc, f1, hamm = aso_eval(y_pred, y_test)

record_result([f"daptE6-2", "semi-supervised", "dapt", acc, f1, hamm])

evaluate_dataset(transformer, classifier, test_instances)

In [ ]:
%%time
transformer, classifier = create_model(transformer, classifier)
optimizer = torch.optim.AdamW(get_param(transformer, classifier), lr = LEARNING_RATE, weight_decay = REGULARIZATION)

num_train_instances = len(train_instances)
num_train_steps = int(num_train_instances / BATCH_SIZE * EPOCH)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)
scheduler = get_constant_schedule_with_warmup(optimizer, num_warmup_steps = num_warmup_steps)

info = train_model(transformer, classifier, optimizer, scheduler, num_epochs=EPOCH, last_epoch=0, last_acc=0)

y_pred, y_test = aso_pred(transformer, classifier, test_instances)
acc, f1, hamm = aso_eval(y_pred, y_test)

record_result([f"daptE6-3", "semi-supervised", "dapt", acc, f1, hamm])

evaluate_dataset(transformer, classifier, test_instances)

In [ ]:
%%time
transformer, classifier = create_model(transformer, classifier)
optimizer = torch.optim.AdamW(get_param(transformer, classifier), lr = LEARNING_RATE, weight_decay = REGULARIZATION)

num_train_instances = len(train_instances)
num_train_steps = int(num_train_instances / BATCH_SIZE * EPOCH)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)
scheduler = get_constant_schedule_with_warmup(optimizer, num_warmup_steps = num_warmup_steps)

info = train_model(transformer, classifier, optimizer, scheduler, num_epochs=EPOCH, last_epoch=0, last_acc=0)

y_pred, y_test = aso_pred(transformer, classifier, test_instances)
acc, f1, hamm = aso_eval(y_pred, y_test)

record_result([f"daptE6-4", "semi-supervised", "dapt", acc, f1, hamm])

evaluate_dataset(transformer, classifier, test_instances)

In [29]:
%%time
transformer, classifier = create_model(transformer, classifier)
optimizer = torch.optim.AdamW(get_param(transformer, classifier), lr = LEARNING_RATE, weight_decay = REGULARIZATION)

num_train_instances = len(train_instances)
num_train_steps = int(num_train_instances / BATCH_SIZE * EPOCH)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)
scheduler = get_constant_schedule_with_warmup(optimizer, num_warmup_steps = num_warmup_steps)

info = train_model(transformer, classifier, optimizer, scheduler, num_epochs=EPOCH, last_epoch=0, last_acc=0)

y_pred, y_test = aso_pred(transformer, classifier, test_instances)
acc, f1, hamm = aso_eval(y_pred, y_test)

record_result([f"daptE6-5", "semi-supervised", "dapt", acc, f1, hamm])

evaluate_dataset(transformer, classifier, test_instances)

Some weights of the model checkpoint at Hendrico/cpt-indobert-large-8 were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch 1 / 100


Val: 100%|██████████| 40/40 [00:10<00:00,  3.67it/s, loss=8.11]


Accuracy: 0.1597444089456869, Precision (micro): 0.19169329073482427, Recall (micro): 0.16806722689075632
Epoch 2 / 100


Val: 100%|██████████| 40/40 [00:10<00:00,  3.64it/s, loss=6.28]


...

Val: 100%|██████████| 40/40 [00:11<00:00,  3.63it/s, loss=nan]


Accuracy: 0.10223642172523961, Precision (micro): 0.1182108626198083, Recall (micro): 0.10364145658263306
Epoch 91 / 100


Evaluate: 100%|██████████| 59/59 [00:07<00:00,  7.84it/s]

================ Classification Report ================
                   precision    recall  f1-score   support

             Anak       1.00      0.93      0.96        43
            Bedah       0.74      0.97      0.84        29
             Gigi       1.00      1.00      1.00        42
             Gizi       0.81      0.98      0.88        43
          Jantung       0.75      0.30      0.43        10
             Jiwa       0.98      0.93      0.95        43
        Kandungan       0.91      0.98      0.94        43
Kulit dan Kelamin       0.88      0.93      0.91        46
             Mata       1.00      1.00      1.00        26
             Paru       0.00      0.00      0.00         2
   Penyakit Dalam       0.70      0.88      0.78        42
            Saraf       1.00      0.14      0.25        14
              THT       0.86      1.00      0.92        43
           Tulang       0.87      0.97      0.92        35
          Urologi       0.89      0.57      0.70        14


/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import os, signal

os.kill(os.getpid(), signal.SIGKILL)